In [1]:
import pandas as pd
import numpy as np
import json

from numpy import savetxt
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import sys
np.set_printoptions(threshold=sys.maxsize)
import os
import matplotlib.pyplot as plt
from sklearn import datasets
import glob

from datetime import datetime

def days(d1, d2):
    d1 = datetime.strptime(d1, "%m/%d/%Y")
    d2 = datetime.strptime(d2, "%m/%d/%Y")
    return abs((d2 - d1).days)+1

In [2]:
f = open('../configs/data_cfg/data_cfg.json')
result_path = json.load(f)['result_path']

In [3]:
numpy_vars = {}
i = 0
for np_name in glob.glob(result_path+'plotweek55result/*.npz'):
    numpy_vars[i] = np.load(np_name)['prediction']
    i = i +1
ML_ensemble = np.stack([numpy_vars[i] for i in range(10)], axis = 0)
ML_ensemble = np.squeeze(ML_ensemble,axis=2)
#ML_ensemble = ML_ensemble.reshape(10,4,50,3)
print(ML_ensemble.shape)
ML_ensemble = np.mean(ML_ensemble,axis=0)
print(ML_ensemble.shape)

(10, 4, 50, 3)
(4, 50, 3)


generate analysis result (MAE, RMSE, MIS, total death)

In [4]:
ML_ensemble[0,1]

array([-203.17221   ,   -0.47118178,  118.79319   ], dtype=float32)

In [5]:
ML_ensemble.shape

(4, 50, 3)

In [6]:
ML_ensemble_list = []
for i in range (4):
    range_low = ML_ensemble[i,:,1:2] - np.dot(ML_ensemble[i,:,1:2] - ML_ensemble[i,:,0:1],
                                 np.array([[2.576,1.96,1.645,1.282,1.036,0.842,0.674,0.524,0.385,0.253,0.126]]))/1.96

    range_high = ML_ensemble[i,:,1:2] + np.dot(ML_ensemble[i,:,2:3] - ML_ensemble[i,:,1:2],
                                 np.array([[0.126,0.253,0.385,0.524,0.674,0.842,1.036,1.282,1.645,1.96,2.576]]))/1.96
    
    tmp_ML_allquantile = np.concatenate([range_low,ML_ensemble[i,:,1:2],range_high],axis=1)
    ML_ensemble_list.append(tmp_ML_allquantile)
    
ML_ensemble_new = np.stack(ML_ensemble_list,axis=0)
print(ML_ensemble_new.shape)
    

(4, 50, 23)


In [7]:
ML_ensemble_new[0,1]

array([-266.87825575, -203.17221633, -170.59526435, -133.0542054 ,
       -107.61315718,  -87.54989152,  -70.17551713,  -54.66268285,
        -40.28745642,  -26.63616226,  -13.50196257,   -0.47118178,
          7.19581368,   14.9236583 ,   22.95574878,   31.41378345,
         40.541159  ,   50.76381961,   62.56855865,   77.53745454,
         99.62570336,  118.793192  ,  156.2762809 ])

number of crossing

In [8]:
np.sum(np.diff(ML_ensemble_new,axis=2) <= 0)

0

solve crossing problem by setting number larger

In [9]:
# ML_diff = np.diff(ML_ensemble_new,axis=2)
# ML_diff = np.maximum(ML_diff,0)
# ML_diff = np.concatenate([ML_ensemble_new[...,:1],ML_diff],axis=2)
# ML_new_ensemble = np.cumsum(ML_diff,axis=2)

# print(ML_diff.shape)
# print(ML_new_ensemble.shape)

# np.sum(np.diff(ML_new_ensemble,axis=2) ==0)

# np.save('ensembledata.npy', ML_new_ensemble)

In [10]:
np.sum(np.diff(ML_ensemble_new,axis=2) ==0)

0

In [11]:
np.save(result_path+'ensembledata.npy', ML_ensemble_new)